In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
# Define portfolio allocation
portfolio_allocation = {
    'BTC-USD': 0.25,  # Bitcoin 25%
    'ETH-USD': 0.15,  # Ethereum 15%
    'SOL-USD': 0.10,  # Solana 10%
    'DOT-USD': 0.10,  # Polkadot 10%
    'LINK-USD': 0.10, # Chainlink 10%
    'Stablecoin': 0.10, # Stablecoin allocation (USDC/DAI)
    'DeFi Yield': 0.15  # DeFi allocation
}

# Fetch historical price data for crypto assets using yfinance
assets = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'DOT-USD', 'LINK-USD']
data = yf.download(assets, start="2021-01-01", end="2023-01-01")['Adj Close']

# Display the head of the data to confirm
print(data.head())


[*********************100%***********************]  5 of 5 completed

Ticker                          BTC-USD    DOT-USD      ETH-USD   LINK-USD  \
Date                                                                         
2021-01-01 00:00:00+00:00  29374.152344   8.306819   730.367554  11.872555   
2021-01-02 00:00:00+00:00  32127.267578   9.208837   774.534973  12.220137   
2021-01-03 00:00:00+00:00  32782.023438  10.033283   975.507690  13.650172   
2021-01-04 00:00:00+00:00  31971.914062   9.469611  1040.233032  13.571063   
2021-01-05 00:00:00+00:00  33992.429688   9.701656  1100.006104  14.539868   

Ticker                      SOL-USD  
Date                                 
2021-01-01 00:00:00+00:00  1.842084  
2021-01-02 00:00:00+00:00  1.799275  
2021-01-03 00:00:00+00:00  2.161752  
2021-01-04 00:00:00+00:00  2.485097  
2021-01-05 00:00:00+00:00  2.157217  


In [12]:
assets = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'DOT-USD', 'LINK-USD']
data = yf.download(assets, start="2021-01-01", end="2023-01-01")
data.to_csv('crypto_data.csv')

[*********************100%***********************]  5 of 5 completed


In [4]:
# Calculate daily returns
returns = data.pct_change().dropna()

# Display the first few rows of returns
print(returns.head())


Ticker                      BTC-USD   DOT-USD   ETH-USD  LINK-USD   SOL-USD
Date                                                                       
2021-01-02 00:00:00+00:00  0.093726  0.108588  0.060473  0.029276 -0.023239
2021-01-03 00:00:00+00:00  0.020380  0.089528  0.259475  0.117023  0.201457
2021-01-04 00:00:00+00:00 -0.024712 -0.056180  0.066350 -0.005795  0.149575
2021-01-05 00:00:00+00:00  0.063197  0.024504  0.057461  0.071388 -0.131938
2021-01-06 00:00:00+00:00  0.083311  0.040654  0.097369  0.179877 -0.105692


In [5]:
# Calculate Annual Percentage Rate (APR)
mean_returns = returns.mean()
apr = mean_returns * 252  # Annualize mean daily returns (252 trading days)

# Calculate Annual Percentage Yield (APY) considering daily compounding
apy = (1 + mean_returns) ** 252 - 1

# Display APR and APY
print(f"APR:\n{apr}\n")
print(f"APY:\n{apy}\n")


APR:
Ticker
BTC-USD    -0.016089
DOT-USD     0.293495
ETH-USD     0.500263
LINK-USD    0.250646
SOL-USD     1.291856
dtype: float64

APY:
Ticker
BTC-USD    -0.015960
DOT-USD     0.340878
ETH-USD     0.648338
LINK-USD    0.284695
SOL-USD     2.627545
dtype: float64



In [6]:
# Calculate volatility (standard deviation) of daily returns
volatility = returns.std() * np.sqrt(252)  # Annualize volatility

# Display volatility
print(f"Annualized Volatility:\n{volatility}\n")


Annualized Volatility:
Ticker
BTC-USD     0.603134
DOT-USD     1.023326
ETH-USD     0.810419
LINK-USD    1.005138
SOL-USD     1.187257
dtype: float64



In [7]:
# Calculate 95% Value at Risk (VaR)
confidence_level = 0.05
z_score = norm.ppf(1 - confidence_level)

# VaR at 95% confidence level
var_95 = z_score * returns.std() * np.sqrt(252)

# Display VaR
print(f"Value at Risk (95% confidence level):\n{var_95}\n")


Value at Risk (95% confidence level):
Ticker
BTC-USD     0.992067
DOT-USD     1.683221
ETH-USD     1.333021
LINK-USD    1.653306
SOL-USD     1.952864
dtype: float64



In [8]:
# Calculate beta for each asset relative to Bitcoin
btc_returns = returns['BTC-USD']
beta = returns.cov().iloc[:, 0] / returns['BTC-USD'].var()

# Display Beta
print(f"Beta relative to Bitcoin:\n{beta}\n")


Beta relative to Bitcoin:
Ticker
BTC-USD     1.000000
DOT-USD     1.205961
ETH-USD     1.094283
LINK-USD    1.182597
SOL-USD     1.069467
Name: BTC-USD, dtype: float64



In [9]:
# Portfolio returns
weights = np.array(list(portfolio_allocation.values())[:-2])  # Exclude DeFi and Stablecoin for now
portfolio_returns = returns.dot(weights)

# Portfolio performance metrics
portfolio_apr = portfolio_returns.mean() * 252
portfolio_apy = (1 + portfolio_returns.mean()) ** 252 - 1
portfolio_volatility = portfolio_returns.std() * np.sqrt(252)
portfolio_var_95 = z_score * portfolio_returns.std() * np.sqrt(252)

# Display portfolio metrics
print(f"Portfolio APR: {portfolio_apr}")
print(f"Portfolio APY: {portfolio_apy}")
print(f"Portfolio Volatility: {portfolio_volatility}")
print(f"Portfolio VaR (95% confidence): {portfolio_var_95}")


Portfolio APR: 0.2442786579936694
Portfolio APY: 0.2765489927568938
Portfolio Volatility: 0.5198312098628132
Portfolio VaR (95% confidence): 0.8550462509454201


In [10]:
# Simulate 30% market crash
crash_factor = 0.70
crash_returns = returns * crash_factor
crash_portfolio_returns = crash_returns.dot(weights)

# Calculate portfolio metrics after crash
crash_apr = crash_portfolio_returns.mean() * 252
crash_volatility = crash_portfolio_returns.std() * np.sqrt(252)
crash_var_95 = z_score * crash_portfolio_returns.std() * np.sqrt(252)

# Display crash scenario metrics
print(f"After 30% Market Crash:")
print(f"Portfolio APR: {crash_apr}")
print(f"Portfolio Volatility: {crash_volatility}")
print(f"Portfolio VaR (95% confidence): {crash_var_95}")


After 30% Market Crash:
Portfolio APR: 0.17099506059556863
Portfolio Volatility: 0.3638818469039687
Portfolio VaR (95% confidence): 0.5985323756617933


In [11]:
# Visualizing with Plotly
import plotly.graph_objects as go

# Portfolio Allocation Pie Chart
labels = list(portfolio_allocation.keys())
values = list(portfolio_allocation.values())

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.3)])
fig.update_layout(title="Portfolio Allocation")
fig.show()

# Daily Returns Plot
fig = px.line(portfolio_returns, title="Portfolio Daily Returns")
fig.show()
